In [2]:
import map_maker

ModuleNotFoundError: No module named 'scipy'

In [1]:
map_maker.make_action_list(map_maker.CONNECTDICT)

NameError: name 'map_maker' is not defined

In [ ]:
qnet_starttensor([
        [1.0000, 1.0000, 1.0000, 0.5000, 1.0000],
        [0.0000, 1.0000, 0.0000, 0.5000, 1.0000],
        [0.0000, 1.0000, 0.0000, 0.5000, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.0000, 1.3333, 2.0000],

        [0.0000, 0.0000, 0.0000, 0.5000, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]
        ])
qnet_desttensor(
        [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
qnet_start_gtensor([ 2., 38.,  3., 98., 39., 37., 68.,  1., 97.,  8.])
qnet_dest_gtensor([10., 44., 70.,  9.,  4., 45., 76., 11., 16., 71.])

[[1.  1.  1.  1.  1. ]
 [0.  1.  0.  1.  1. ]
 [0.  1.  0.  1.  1. ]
 [0.  0.5 0.  0.5 1. ]
 [0.  0.5 0.  0.5 1. ]
 [0.  1.  0.  1.  1. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]

 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]]
gnn_from_start[96 36  1 37 67]
gnn_to_start[36  0 66  6 41]
gnn_from_dest[10 44 70  9  4]
gnn_to_dest[45 76 11 16 71]

In [ ]:

class QNet(nn.Module):
    def __init__(self, state_dim:int ,action_dims:int) -> None:
        super().__init__()
        self.state_dims = state_dim
        self.action_dims = action_dims

        self.node_data_dim = NODE_DATA_DIM
        self.startnode_dims = NODE_DATA_DIM * NUM_NODES_OF_LOCAL_GRAPH_START
        self.destnode_dims = NODE_DATA_DIM * NUM_NODES_OF_LOCAL_GRAPH_DEST
        self.fromnodes = NUM_NODES_OF_LOCAL_GRAPH_START
        self.tonodes = NUM_NODES_OF_LOCAL_GRAPH_DEST
        self.elsedim = 2

        self.singledims = self.elsedim + self.node_data_dim * (self.action_dims + 1)
        self.connect_dims = self.singledims * KNN_AGENTS
        
        if SAME_GNN_MODEL:
            self.state_graph_l1 = LocalGraphEmbedding(input_dim=self.node_data_dim, output_dim=self.node_data_dim, connect=CONNECTDICT) 
            self.dest_graph_l1 = LocalGraphEmbedding(input_dim=self.node_data_dim, output_dim=self.node_data_dim, connect=CONNECT_TO_DICT)
        else:
            self.state_graph_l1 = [LocalGraphEmbedding(input_dim=self.node_data_dim, output_dim=self.node_data_dim, connect=CONNECTDICT) for _ in range(KNN_AGENTS)]
            self.dest_graph_l1 = [LocalGraphEmbedding(input_dim=self.node_data_dim, output_dim=self.node_data_dim, connect=CONNECT_TO_DICT) for _ in range(KNN_AGENTS)]

        
        self.dims_l1 = 512
        self.dims_before_last = self.dims_l1
        self.l1 = nn.Linear(self.connect_dims, self.dims_l1)
        nn.init.kaiming_uniform_(self.l1.weight, mode="fan_in", nonlinearity="relu")
        
        self.l_last = FactorizedNoisy(self.dims_before_last, action_dims)
        self.ladv_last = FactorizedNoisy(self.dims_before_last, action_dims)
        self.lv_last = FactorizedNoisy(self.dims_before_last, 1)
        
        self.flatten = nn.Flatten()
    
    def find_indices(self, actionlist, nodes_float):
        indices = []
        
        nodes = nodes_float.clone().detach().to(torch.int32).to('cpu').numpy()
        for item in actionlist[nodes[0]]:
            if item in nodes:
                indices.append(np.where(nodes==item)[0][0])
        return indices

    def forward(self, x):
        batch_num = x.shape[0]
        x_reshape = torch.reshape(x, (batch_num, KNN_AGENTS, -1))
        x_split = torch.split(x_reshape, (self.startnode_dims,self.destnode_dims,self.fromnodes,self.tonodes, self.elsedim), dim=2)
        x_start_infos = torch.reshape(x_split[0], (batch_num, KNN_AGENTS, self.fromnodes, -1))
        x_dest_infos = torch.reshape(x_split[1], (batch_num, KNN_AGENTS, self.tonodes, -1))
        x_start_graphs = x_split[2]
        x_dest_graphs = x_split[3]
        x_elses = x_split[4]

        x_feat = []
        for batch in range(batch_num):
            x_start_info = x_start_infos[batch]
            x_dest_info = x_dest_infos[batch]
            x_start_graph = x_start_graphs[batch]
            x_dest_graph = x_dest_graphs[batch]
            x_else = x_elses[batch]
            x_tmp_feat = []
            for robot in range(KNN_AGENTS):
                x_start_info_robot = x_start_info[robot]
                x_dest_info_robot = x_dest_info[robot]
                x_start_graph_robot = x_start_graph[robot]
                x_dest_graph_robot = x_dest_graph[robot]
                x_else_robot = x_else[robot]
                
                if SAME_GNN_MODEL:
                    x_start_info_robot = self.state_graph_l1(x_start_info_robot, x_start_graph_robot)
                    x_dest_info_robot = self.dest_graph_l1(x_dest_info_robot, x_dest_graph_robot)
                else:
                    x_start_info_robot = self.state_graph_l1[robot](x_start_info_robot, x_start_graph_robot)
                    x_dest_info_robot = self.dest_graph_l1[robot](x_dest_info_robot, x_dest_graph_robot)
                
                rows = self.find_indices(ACTION_LIST, x_start_graph_robot)
                x_tmp_feat.append(torch.cat([torch.flatten(x_start_info_robot[rows]), x_dest_info_robot[0], x_else_robot]))
            x_tmp_feat = torch.cat(x_tmp_feat)
            x_feat.append(x_tmp_feat)
            
        x = torch.cat(x_feat)
        x = torch.reshape(x, (batch_num, -1))

        x = F.relu(self.l1(x))
        if DIM2:
            x = F.relu(self.l2(x))

        if DUELINGMODE:
            adv = self.ladv_last(x)
            v = self.lv_last(x)
            averagea = adv.mean(1, keepdim=True)
            return v.expand(-1, self.action_dims) + (adv - averagea.expand(-1, self.action_dims))
        else:
            x = self.l_last(x)
            return x

In [1]:
import networkx as nx

def dict_to_networkx(adj_list):
    G = nx.DiGraph()
    for node, neighbors in adj_list.items():
        for neighbor in neighbors:
            G.add_edge(node, neighbor)
    return G

In [2]:
def networkx_to_dict(G):
    adj_list = {node: list(G.neighbors(node)) for node in G.nodes()}
    return adj_list

In [3]:
# 辞書型の隣接リストを定義
adj_list = {
    1: [2, 3],
    2: [3],
    3: [1],
    4: [2, 3]
}

# 辞書型の隣接リストからnetworkxグラフへ変換
G = dict_to_networkx(adj_list)

# networkxグラフから辞書型の隣接リストへ変換
converted_adj_list = networkx_to_dict(G)

print("変換されたnetworkxグラフ:", G.edges())
print("元に戻された辞書型の隣接リスト:", converted_adj_list)


変換されたnetworkxグラフ: [(1, 2), (1, 3), (2, 3), (3, 1), (4, 2), (4, 3)]
元に戻された辞書型の隣接リスト: {1: [2, 3], 2: [3], 3: [1], 4: [2, 3]}


In [ ]:
tensor([[[[1.0000e+00, 1.0000e+00, 1.0000e+00, 3.3333e-01, 7.9800e+02, 4.7600e+02],
          [0.0000e+00, 1.0000e+00, 0.0000e+00, 3.3333e-01, 7.3200e+02, 4.7600e+02],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.6667e-01, 7.3200e+02, 5.3000e+02]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]]],


        [[[1.0000e+00, 1.0000e+00, 1.0000e+00, 1.6667e-01, 9.9700e+02, 6.3600e+02],
          [0.0000e+00, 1.0000e+00, 0.0000e+00, 1.6667e-01, 1.0630e+03, 6.3600e+02],
          [0.0000e+00, 1.0000e+00, 0.0000e+00, 1.6667e-01, 1.0630e+03, 5.8300e+02]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]]],


        [[[1.0000e+00, 1.0000e+00, 1.0000e+00, 1.6667e-01, 7.3200e+02,
           5.3000e+02],
          [0.0000e+00, 1.0000e+00, 0.0000e+00, 5.0000e-01, 7.9800e+02,
           5.3000e+02],
          [0.0000e+00, 1.0000e+00, 0.0000e+00, 3.3333e-01, 7.9800e+02,
           4.7600e+02]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]]],


        ...,


        [[[1.0000e+00, 1.0000e+00, 1.0000e+00, 1.6667e-01, 1.0630e+03,
           6.3600e+02],
          [0.0000e+00, 1.0000e+00, 0.0000e+00, 1.6667e-01, 1.0630e+03,
           5.8300e+02],
          [0.0000e+00, 1.0000e+00, 0.0000e+00, 1.6667e-01, 9.9700e+02,
           5.8300e+02]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]]],


        [[[1.0000e+00, 1.0000e+00, 1.0000e+00, 1.6667e-01, 9.9700e+02,
           5.8300e+02],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.9700e+02,
           6.3600e+02],
          [0.0000e+00, 1.0000e+00, 0.0000e+00, 1.6667e-01, 9.3100e+02,
           5.8300e+02]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]]],


        [[[1.0000e+00, 1.0000e+00, 1.0000e+00, 3.3333e-01, 1.0630e+03,
           5.8300e+02],
          [0.0000e+00, 1.0000e+00, 0.0000e+00, 3.3333e-01, 9.9700e+02,
           5.8300e+02],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.9700e+02,
           6.3600e+02]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]]]], device='cuda:0')